<a href="https://colab.research.google.com/github/gabrielpetersson/flan-t5-tweet-quality-predictor/blob/master/tweet_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required libraries
from google.colab import auth
from google.colab import drive
import pandas as pd

auth.authenticate_user()
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [3]:
!ls /gdrive/MyDrive/ai

file_path = '/gdrive/MyDrive/ai/tweets/bigboy.csv'
full_df = pd.read_csv(file_path)


13B  7B  tokenizer_checklist.chk  tokenizer.model  tweets


In [4]:
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install rich[jupyter]
!pip install numba

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
short_df = full_df #.iloc[:16_000_000]

In [6]:
df = short_df
print("initial length", len(df))

df = short_df.drop(index=df[df["lang"] != "en"].index)
print("eng length", len(df))

df = df.drop_duplicates(subset=["full_text"])
print("drop tweet duplicates", len(df))

user_name_counts = df['user_name'].value_counts()
drop_users = user_name_counts[user_name_counts >= 500].index.tolist()
df = df[~df['user_name'].isin(drop_users)]
print("drop spammers", len(df), len(drop_users))

drop_users = user_name_counts[user_name_counts <= 10].index.tolist()
df = df[~df['user_name'].isin(drop_users)]
print("drop low output users", len(df), len(drop_users))

total_likes = df.groupby('user_name')['likes'].sum()
row_count = df.groupby('user_name').size()
average_likes_per_row = total_likes / row_count
users_meeting_condition = average_likes_per_row[average_likes_per_row > 5].index.tolist()
df = df[df['user_name'].isin(users_meeting_condition)]
print("drop low likes ppl", len(df), len(users_meeting_condition))

n = df["creator_followers_count"] >= 500
df = df[n]
print("drop low followers", len(df))

n = df["likes"] >= 10
df = df[n]
print("drop low likes tweetes", len(df))

initial length 4216599
eng length 3727473
drop tweet duplicates 2646476
drop spammers 2545668 92
drop low output users 1036233 850258
drop low likes ppl 231183 7327
drop low followers 157675
drop low likes tweetes 80093


In [7]:
def normalize_score(group):
    group['normalized_score'] = group['likes'] * 100 / group['creator_followers_count']
    
    min_score = group['normalized_score'].min()
    max_score = group['normalized_score'].max()
    
    if max_score == min_score:
       group['normalized_score'] = 1
    else:
        group['normalized_score'] = (group['normalized_score'] - min_score) / (max_score - min_score)
    return group

df = df.groupby('user_name').apply(normalize_score)
#df['scaled_score'] = round(df['normalized_score'] * (50 - 1) + 1)
#df['scaled_score'] = df['scaled_score'].astype(int)
df['full_text'] = df['full_text'].str.replace("@\w+", "[USER]")
df['full_text'] = df['full_text'].str.replace("#\w+", "[HASHTAG]")
#df["text_score"] = df["scaled_score"].astype(str) 

df.head(1000)

<ipython-input-7-4279dae3fdd5>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['full_text'] = df['full_text'].str.replace("@\w+", "[USER]")
<ipython-input-7-4279dae3fdd5>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['full_text'] = df['full_text'].str.replace("#\w+", "[HASHTAG]")


,number,id,url,full_text,hashtags,reply_count,lang,user_name,likes,publicationtime,creator_followers_count,normalized_score
366,3,1455580584693014529,https://twitter.com/LMKMovieManiac/status/1455580584693014529,"This Diwali day, chill with the [HASHTAG] musical teaser; 4th Nov @ 11:11am 👍\n \nIn Cinemas Soon.\n\n[USER] [USER] [USER] [USER] [USER] [USER] [USER] [USER] https://t.co/Q1FB6hwn6x",SuperMachi,1,en,LMKMovieManiac,57,Tue Nov 02 17:00:29 +0000 2021,370835,1.809729e-03
1461,72,1455615086865915907,https://twitter.com/KristenBarbar/status/1455615086865915907,"Black describes Rittenhouse minutes after the shooting: He was freaking out. Really scared, pale, sweating. He just said he had to do it, it was self-defense, people were trying to hurt him. [HASHTAG]",KyleRittenhouse,6,en,KristenBarbar,25,Tue Nov 02 19:17:35 +0000 2021,1667,6.641326e-02
1477,88,1455614067708764160,https://twitter.com/Andrew_Havranek/status/1455614067708764160,"Black said he never saw Rosenbaum, Huber, or Grosskreutz that night. \n\nThose are the three men [HASHTAG] shot.",KyleRittenhouse,5,en,Andrew_Havranek,11,Tue Nov 02 19:13:32 +0000 2021,2006,1.054529e-03
2616,11,1455617559215153159,https://twitter.com/asportstvpk/status/1455617559215153159,Congrats [HASHTAG] on Reaching the Semi-Finals! [HASHTAG]\n Watch the post-match analysis at [HASHTAG] LIVE on [HASHTAG] [HASHTAG] \n[HASHTAG] [HASHTAG] [HASHTAG] https://t.co/9FScajdvzZ,"TeamGreen,ICCT20WorldCup2021,ThePavilion,ASportsHD,PakistansfirstHDsportschannel,LiveTheGame,PakvsNamibia,PAKvsNAM",0,en,asportstvpk,90,Tue Nov 02 19:27:24 +0000 2021,41508,8.220978e-02
2658,53,1455605348866396167,https://twitter.com/asportstvpk/status/1455605348866396167,"Happiness everywhere, Congratulations 🎉🥳 [HASHTAG] on reaching the [HASHTAG] of [HASHTAG]\n \nWatch the post-match analysis at [HASHTAG] LIVE on [HASHTAG] [HASHTAG] \n[HASHTAG] [HASHTAG] [HASHTAG] [HASHTAG] https://t.co/igAZmfDoLD","Pakistan,SemiFinals,ICCT20WorldCup2021,ThePavilion,ASportsHD,PakistansfirstHDsportschannel,LiveTheGame,PakvsNamibia,PAKvsNAM,askthepavilion",20,en,asportstvpk,448,Tue Nov 02 18:38:53 +0000 2021,41508,5.726513e-01
3787,96,1455593619532955652,https://twitter.com/votesbybelieber/status/1455593619532955652,The Collaboration Song Stats for PCAs:\n\n1- Peaches : 29.4K (+3.1K)\n2- L** : 3.8K (+0.2K)\n\nVote on the website too: https://t.co/1IpTIXqtFi\n\n[HASHTAG] [HASHTAG] [HASHTAG] https://t.co/JWuXoaIlJR,"Peaches,TheCollaborationSong,PCAs",1,en,votesbybelieber,12,Tue Nov 02 17:52:17 +0000 2021,1014,5.117985e-03
3826,20,1455623206623084547,https://twitter.com/Hold_ShibaInu/status/1455623206623084547,Today Kraken will be listing the ever anticipated $SHIB AFTER SCHEDULED MAINTENANCE @ 22:00 UTC 6PM EST or 3PM PST\n\nHere's How it Can Impact [HASHTAG] Price\n\n[USER] STATISTICS BEFORE $SHIBA LISTINGS\n\n$1.61 BILLION 24 HOUR VOLUME🚀\n\n[HASHTAG] [HASHTAG] [HASHTAG] https://t.co/oBNCtE2wAs,"ShibainuCoin,SHIB,shiba,SHIBARMY",0,en,Hold_ShibaInu,11,Tue Nov 02 19:49:51 +0000 2021,4471,0.000000e+00
4312,29,1455619451521470472,https://twitter.com/SajSadiqCricket/status/1455619451521470472,"Namibia Coach Pierre de Bruyn ""it was amazing for the batters to experience that. When we qualified for the Super 12 they knew it was coming &amp; they saw it tonight. They can take so much confidence after the batting performance getting 144 against that world-class attack"" [HASHTAG]",PAKvNAM,3,en,SajSadiqCricket,654,Tue Nov 02 19:34:56 +0000 2021,343236,0.000000e+00
4328,45,1455617328318713856,https://twitter.com/cricketpakcompk/status/1455617328318713856,Imran Tahir lauds ‘dangerous’ Pakistan\n\nHe also claimed that Pakistan and South Africa will be the finalists of the ongoing T20 World Cup\n\nRead more: https://t.co/VdAUKWOXty\n\n[HASHTAG] [HASHTAG] [USER] [USER] https://t.co/tG1DYuqD8J,"T20WorldCup,PAKvNAM",1,en,cricketpakcompk,67,Tue Nov 02 19:26:29 +0000 2021,33791,1.073000e-01
4343,60,1455615547421429761,https://twitter.com/khelshel/status/145561

In [ ]:
import torch
import random
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW, AutoModelForSeq2SeqLM, AutoTokenizer

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau


class T5ForRegression(T5ForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)
        self.regression_head = torch.nn.Linear(config.d_model, 1)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        head_mask=None,
        inputs_embeds=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        encoder_outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        last_hidden_state = encoder_outputs.last_hidden_state
        regression_output = self.regression_head(last_hidden_state[:, -1])

        if labels is not None:
            loss = torch.nn.MSELoss()(regression_output.view(-1), labels.view(-1))
            return (loss, regression_output) if return_dict else (loss, regression_output, None)
        else:
            return regression_output if return_dict else (regression_output, None)

class CustomDataset(Dataset):
    def __init__(self, tokenizer, texts, labels, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(self.texts[idx], max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        inputs['input_ids'] = inputs['input_ids'].squeeze()
        inputs['attention_mask'] = inputs['attention_mask'].squeeze()
        inputs['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return inputs

sh = df #.iloc[:5000]
texts = sh["full_text"].tolist()
labels = sh["normalized_score"].tolist()
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.1, random_state=42)

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForRegression.from_pretrained("google/flan-t5-base")
train_dataset = CustomDataset(tokenizer, train_texts, train_labels)
val_dataset = CustomDataset(tokenizer, val_texts, val_labels)

log_steps = 200
batch = 48
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=batch,
    per_device_eval_batch_size=batch,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=log_steps,
    logging_dir="./logs",
    logging_steps=log_steps,
    logging_strategy="steps",
    learning_rate=5e-6,
    weight_decay=0.01,
    warmup_steps=50,
    seed=42,
)

def compute_metrics(eval_pred):
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.squeeze()
    rmse = mean_squared_error(labels, preds, squared=False)
    return {"rmse": rmse}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

model.save_pretrained("./fine-tuned-regression-model")
tokenizer.save_pretrained("./fine-tuned-regression-model")

Some weights of T5ForRegression were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['regression_head.weight', 'regression_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Rmse
100,0.111700,0.087744,0.296216
200,0.084100,0.086897,0.294784
300,0.082700,0.085258,0.291989
400,0.088300,0.084676,0.290992
500,0.090300,0.084714,0.291057
600,0.086400,0.083700,0.289309
700,0.085300,0.083411,0.288810
800,0.083900,0.083194,0.288433
900,0.084000,0.083042,0.288169
1000,0.088200,0.083201,0.288445


In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [ ]:
import torch
from transformers import T5Tokenizer

model = T5ForRegression.from_pretrained("./fine-tuned-regression-model")
tokenizer = T5Tokenizer.from_pretrained("./fine-tuned-regression-model")
model.eval()

def make_prediction(text):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt", max_length=512, padding="max_length", truncation=True)
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]

        output = model(input_ids=input_ids, attention_mask=attention_mask)
        prediction = output[0].item()

    return prediction

texts = ["Your text here", "Here is a great tweet!", "Abortion is dumb", "Abortion is great", "Abortion is dumb [USER]"]
for text in texts:
    prediction = make_prediction(text)
    print(f"Tweet: {text}, Prediction: {prediction}")